# GPTアクションライブラリ: BigQuery

## はじめに

このページでは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。作業を進める前に、まず以下の情報をよく理解しておいてください：

- [GPT Actionsの紹介](https://platform.openai.com/docs/actions)
- [GPT Actionsライブラリの紹介](https://platform.openai.com/docs/actions/actions-library)
- [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

この特定のGPT Actionは、GoogleクラウドのAnalytical Data Warehouseである**Google BigQuery**への接続方法の概要を提供します。このActionは、ユーザーの質問を受け取り、関連するテーブルをスキャンしてデータスキーマを収集し、その後ユーザーの質問に答えるためのSQLクエリを作成します。

注意：これらの手順は、結果そのものではなく、機能するSQL文を返します。現在、CSVファイルを返すためにはミドルウェアが必要です - その例についての手順は近日中に投稿予定です。

### 価値 + ビジネス活用事例

**価値**: ユーザーはChatGPTの自然言語機能を活用して、BigQueryのDWHに直接接続できるようになりました。

**使用例**: 
- データサイエンティストはテーブルに接続し、ChatGPTのData Analysisを使用してデータ分析を実行できます
- 一般的なデータユーザーは、トランザクションデータに対して基本的な質問をすることができます
- ユーザーは自分のデータと潜在的な異常値についてより多くの可視性を得ることができます

## アプリケーション情報

### アプリケーションキーリンク

開始する前に、アプリケーションの以下のリンクを確認してください：
- アプリケーションWebサイト: https://cloud.google.com/bigquery 
- アプリケーションAPI ドキュメント: https://cloud.google.com/bigquery/docs/reference/rest

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行してください：
- GCPプロジェクトを設定する
- そのGCPプロジェクト内にBQデータセットを設定する
- ChatGPT経由でBigQueryに認証するユーザーが、そのBQデータセットにアクセスできることを確認する

## ChatGPT ステップ

### カスタムGPT指示

Custom GPTを作成したら、以下のテキストをInstructionsパネルにコピーしてください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
**Context**: You are an expert at writing BigQuery SQL queries. A user is going to ask you a question. 

**Instructions**:
1. No matter the user's question, start by running `runQuery` operation using this query: "SELECT column_name, table_name, data_type, description FROM `{project}.{dataset}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`" 
-- Assume project = "<insert your default project here>", dataset = "<insert your default dataset here>", unless the user provides different values 
-- Remember to include useLegacySql:false in the json output
2. Convert the user's question into a SQL statement that leverages the step above and run the `runQuery` operation on that SQL statement to confirm the query works. Add a limit of 100 rows
3. Now remove the limit of 100 rows and return back the query for the user to see

**Additional Notes**: If the user says "Let's get started", explain that the user can provide a project or dataset, along with a question they want answered. If the user has no ideas, suggest that we have a sample flights dataset they can query - ask if they want you to query that

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
openapi: 3.1.0
info:
  title: BigQuery API
  description: API for querying a BigQuery table.
  version: 1.0.0
servers:
  - url: https://bigquery.googleapis.com/bigquery/v2
    description: Google BigQuery API server
paths:
  /projects/{projectId}/queries:
    post:
      operationId: runQuery
      summary: Executes a query on a specified BigQuery table.
      description: Submits a query to BigQuery and returns the results.
      x-openai-isConsequential: false
      parameters:
        - name: projectId
          in: path
          required: true
          description: The ID of the Google Cloud project.
          schema:
            type: string
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                query:
                  type: string
                  description: The SQL query string.
                useLegacySql:
                  type: boolean
                  description: Whether to use legacy SQL.
                  default: false
      responses:
        '200':
          description: Successful query execution.
          content:
            application/json:
              schema:
                type: object
                properties:
                  kind:
                    type: string
                    example: "bigquery#queryResponse"
                  schema:
                    type: object
                    description: The schema of the results.
                  jobReference:
                    type: object
                    properties:
                      projectId:
                        type: string
                      jobId:
                        type: string
                  rows:
                    type: array
                    items:
                      type: object
                      properties:
                        f:
                          type: array
                          items:
                            type: object
                            properties:
                              v:
                                type: string
                  totalRows:
                    type: string
                    description: Total number of rows in the query result.
                  pageToken:
                    type: string
                    description: Token for pagination of query results.
        '400':
          description: Bad request. The request was invalid.
        '401':
          description: Unauthorized. Authentication is required.
        '403':
          description: Forbidden. The request is not allowed.
        '404':
          description: Not found. The specified resource was not found.
        '500':
          description: Internal server error. An error occurred while processing the request.

## 認証手順

以下は、この3rdパーティアプリケーションでの認証設定に関する手順です。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

### 事前アクション手順

ChatGPTで認証を設定する前に、アプリケーション内で以下の手順を実行してください。

- Google Cloud Consoleに移動する
- API & Services > Credentialsに移動する
- 新しいOAuth認証情報を作成する（または既存のものを使用する）
- OAuth Client IDとClient Secretを見つけて、両方の値を安全に保存する（以下のスクリーンショットを参照）

![gptactions_BigQuery_auth.png](../../../images/gptactions_BigQuery_auth.png)

申し訳ございませんが、翻訳すべきテキストが「### In ChatGPT」の見出しのみのようです。

この見出しを翻訳すると以下のようになります：

### ChatGPTにて

もし他に翻訳したいテキストがございましたら、お聞かせください。

ChatGPTで、「Authentication」をクリックし、**「OAuth」**を選択します。以下の情報を入力してください。

- **Client ID**: 上記の手順で取得したClient IDを使用
- **Client Secret**: 上記の手順で取得したClient Secretを使用
- **Authorization URL**: https://accounts.google.com/o/oauth2/auth
- **Token URL**: https://oauth2.googleapis.com/token 
- **Scope**: https://www.googleapis.com/auth/bigquery 
- **Token**: Default (POST)

### アクション後のステップ

ChatGPTで認証を設定したら、アプリケーション内で以下の手順に従ってActionを完成させてください。

- GPT ActionからコールバックURLをコピーする
- 「認証済みリダイレクトURI」（上記のスクリーンショットを参照）に、コールバックURLを追加する

### FAQ & トラブルシューティング

- *コールバックURLエラー:* ChatGPTでコールバックURLエラーが発生した場合は、上記のスクリーンショットに注意深く従ってください。アクションが正しく認証されるように、コールバックURLを直接GCPに追加する必要があります
- *スキーマが間違ったプロジェクトやデータセットを呼び出す:* ChatGPTが間違ったプロジェクトやデータセットを呼び出す場合は、以下のいずれかをより明確にするように指示を更新することを検討してください：(a) どのプロジェクト/データセットを呼び出すべきか、または (b) クエリを実行する前にユーザーにそれらの正確な詳細を提供するよう要求する

*私たちに優先してほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。*